In [1]:
import pandas as pd
import os

def find_bug(df):
    """
    df_offers имеет странные дублирующиеся model_id_pred. Векроятно это баг в функции top_n_similar 
    (не находит достаточное количество до n шт матчей + дальнейший convert_columns_to_int)
    """
    columns_to_check = ['model_id_pred_1', 'model_id_pred_2', 'model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
    duplicate_rows_indices = df[df[columns_to_check].apply(lambda row: row.duplicated().sum() >= 1, axis=1)].index
    new_df = df.loc[duplicate_rows_indices]
    return new_df

def merge_columns_to_list(df, n, column_name):
    """
    Функция для объединения значений из нескольких колонок в один список.
    
    Параметры:
        df (DataFrame): Исходный DataFrame.
        n (int): Количество колонок, которые нужно объединить.
        column_name (str): Имя обрабатываемых колонок.
        
    Возвращает:
        DataFrame: DataFrame с добавленным столбцом, содержащим объединенные значения в виде списка.
    """
    # Создаем список колонок для объединения
    columns = [f"{column_name}_{i}" for i in range(1, n+1)]
    
    # Объединяем значения в список
    df[column_name] = df[columns].apply(lambda row: row.tolist(), axis=1)
    
    return df

def exclude_etalon_from_df(df, df_etalon, imya_kategorii):
    df_exclude = df_etalon[df_etalon.category_name == imya_kategorii]
    print(f"{imya_kategorii}: exclude {len(df_exclude)} rows")
    return df[~df.name.isin(df_exclude.name)]

pth_models = "/home/sondors/Documents/price/BERT_data/data/10-04-2024_Timofey/2801_models_Apple.csv"
# pth_models = "/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_models_Apple.csv"

pth_offers = "/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_bsize210_offers_top_n_Apple_test.csv"
# pth_csv_exclude = '/home/sondors/Documents/price/offers_clusterization/data/14_categories_balanced.csv'

id_category = {
        # 3902: 'диктофоны, портативные рекордеры',
        # 510402: 'электронные книги',
        # 4302: 'автомобильные телевизоры, мониторы',
        # 2815: 'смарт-часы и браслеты',
        # 3901: 'портативные медиаплееры',
        # 3904: 'портативная акустика',
        2801: 'мобильные телефоны',
        # 3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
        # 510401: 'планшетные компьютеры и мини-планшеты',
        # 2102: 'наушники, гарнитуры, наушники c микрофоном',
        # 3903: 'радиоприемники, радиобудильники, радиочасы',
        # 3907: 'магнитолы',
        # 280801: 'GPS-навигаторы'
        }

# id_category = {
    # 921201: 'корм для кошек',
    # 963401: 'колбасы',
    # 710502: 'стиральные машины',
    # 977135: 'ботинки, полуботинки',
    # 911906: 'парфюмерия',
    # 976132: 'туфли и лоферы',
    # 740101: 'кухонные мойки',
    # 7106: 'микроволновые печи',
    # 921401: 'корм для собак',
    # 963302: 'пицца'
    # }

df_models = pd.read_csv(pth_models, sep=";")
# df_models = df_models.drop(columns=['average_price', 'comment'])
df_offers = pd.read_csv(pth_offers, sep=";")
df_offers = df_offers[df_offers.model_id != 0]
# df_offers = df_offers.drop(columns=['true_match', 'false_match'])

# df_exclude = pd.read_csv(pth_csv_exclude, sep=';')
# for category in id_category.values():
#     df_new = exclude_etalon_from_df(df_offers, df_exclude, category)

df_offers = merge_columns_to_list(df_offers, 5, "model_id_pred")
df_offers = merge_columns_to_list(df_offers, 5, "similarity")
df_offers


,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,...,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_pred,similarity
0,"Смартфон Apple iPhone 11 128GB Черный ""Хорошее...",7883196,1870506860,False,мобильные телефоны,2801,29990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","Любовь с первого, второго, третьего, четвёртог...",Смартфоны с пробегом Б/У,...,7883196,30.34,7883196,30.34,7883196,30.34,7883196,30.34,"[7883196, 7883196, 7883196, 7883196, 7883196]","[30.34, 30.34, 30.34, 30.34, 30.34]"
1,Смартфон Apple iPhone 14 Pro Max 512Gb 6Gb тем...,7883608,1870801227,True,мобильные телефоны,2801,146389.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple A2896 iPhone 14 Pro Max 512Gb 6...,"Ноутбуки, планшеты, смартфоны :: Смартфоны",...,7883608,25.73,7883604,23.47,7883513,19.47,7883599,18.30,"[7883612, 7883608, 7883604, 7883513, 7883599]","[25.83, 25.73, 23.47, 19.47, 18.3]"
2,Смартфон Apple iPhone 15 Pro 256 Гб Dual SIM т...,7883739,1876282775,True,мобильные телефоны,2801,135880.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Новый инновационный дизайн включает в себя зад...,Цифровая техника :: Смартфоны и мобильные теле...,...,7883741,26.91,7883733,26.88,7883736,25.14,7883640,23.86,"[7883737, 7883741, 7883733, 7883736, 7883640]","[29.88, 26.91, 26.88, 25.14, 23.86]"
3,Смартфон Apple iPhone 14 Plus 256GB eSim Yello...,7883485,1876136258,False,мобильные телефоны,2801,72790.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple iPhone 14 Plus 256GB eSim Yello...,Смартфоны :: iPhone 14,...,7883497,27.53,7883679,25.12,7883484,21.16,7883633,20.00,"[7883485, 7883497, 7883679, 7883484, 7883633]","[30.86, 27.53, 25.12, 21.16, 20.0]"
4,Смартфон Apple iPhone 13 512GB Pink (розовый),7883348,1876413447,False,мобильные телефоны,2801,76990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple iPhone 13 512GB Pink (розовый) ...,Смартфоны :: iPhone 13,...,7883347,23.98,7883659,21.38,7883649,20.45,7883654,17.73,"[7883348, 7883347, 7883659, 7883649, 7883654]","[30.42, 23.98, 21.38, 20.45, 17.73]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3351,Смартфон Apple iPhone 15 Pro Max 256Gb eSIM + ...,7883776,1873612511,False,мобильные телефоны,2801,149990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Аппарат поддерживает 1 физическую SIM и eSIM. ...,Смартфоны :: Apple,...,7883768,26.36,7883740,17.53,7883848,14.82,7883643,14.68,"[7883776, 7883768, 7883740, 7883848, 7883643]","[28.66, 26.36, 17.53, 14.82, 14.68]"
3352,Смартфон Apple iPhone 15 Pro Max 256Gb eSIM on...,7883766,1873611822,False,мобильные телефоны,2801,149990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","Аппарат не поддерживает физические SIM, только...",Смартфоны :: Apple,...,7883774,25.19,7883770,21.66,7883731,20.59,7883429,17.34,"[7883766, 7883774, 7883770, 7883731, 7883429]","[28.41, 25.19, 21.66, 20.59, 17.34]"
3353,Смартфон Apple iPhone 15 Pro 256Gb eSIM + SIM ...,7883739,1873611850,False,мобильные телефоны,2801,135990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Аппарат поддерживает 1 физическую SIM и eSIM. ...,Смартфоны :: Apple,...,7883731,28.05,7883735,26.05,7883729,25.75,7883774,20.91,"[7883739, 7883731, 7883735, 7883729, 7883774]","[30.55, 28.05, 26.05, 25.75, 20.91]"
3354,"Смартфон Apple iPhone 15 Pro 512 ГБ, Dual eSIM...",7883743,1878760256,True,мобильные телефоны,2801,114990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Цвет: Синий титан | Автофокус: Да | Диагональ ...,Смартфоны :: iPhone :: iPhone 15 Pro,...,7883646,24.80,7883691,24.11,7883779,23.44,7883450,21.14,"[7883743, 7883646, 7883691, 7883779, 7883450]","[30.66, 24.8, 24.11, 23.44, 21.14]"


In [2]:
def accuracy_top_n(offer_model_id, offer_model_id_pred, top_k):
    correct_predictions = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        if true_id in top_k_predictions:
            correct_predictions += 1

    accuracy = correct_predictions / len(offer_model_id)
    return accuracy

def calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k): # проверить функцию
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        
        if true_id in top_k_predictions:
            true_positives += 1
        else:
            false_negatives += 1
            false_positives += (true_id in predicted_ids)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

def calc_acc_for_category(df_offers, df_models, imya_kategorii):
    print(f"{imya_kategorii}")
    df_offers = df_offers[df_offers.category_name == imya_kategorii]
    df_models = df_models[df_models.category_name == imya_kategorii]

    offer_names = list(df_offers['name'])
    offer_model_id = list(df_offers['model_id'])
    offer_model_id_pred = list(df_offers['model_id_pred'])

    model_full_names = list(df_models['full_name'])

    print(f"\tофферов {len(offer_model_id)} шт поиск матча для каждого среди {len(model_full_names)} шт моделей:\n")

    top1_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 1)
    top2_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 2)
    top3_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 3)
    top4_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 4)
    top5_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 5)

    top1_precision, top1_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=1)
    top3_precision, top3_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=3)
    top5_precision, top5_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=5)

    print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}')
    print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}')
    print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}\n')

    # print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}, Precision = {top1_precision:.2%}, Recall = {top1_recall:.2%}')
    # print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}, Precision = {top3_precision:.2%}, Recall = {top3_recall:.2%}')
    # print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}, Precision = {top5_precision:.2%}, Recall = {top5_recall:.2%}\n')
    return (top1_accuracy, top2_accuracy, top3_accuracy, top4_accuracy, top5_accuracy)

print(f"{pth_offers.split('/')[-1]}")
for category in id_category.values():
    accuracies_top_n = calc_acc_for_category(df_offers, df_models, category)

2801_bsize210_offers_top_n_Apple_test.csv
мобильные телефоны
	офферов 3356 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 89.99%
	Top-3: Accuracy = 97.94%
	Top-5: Accuracy = 98.09%



In [3]:
"""
мобильные телефоны
	офферов 7193 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 7.66%
	Top-3: Accuracy = 24.69%
	Top-5: Accuracy = 33.16%
"""

"""
мобильные телефоны
	офферов 607 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 41.35%
	Top-3: Accuracy = 67.38%
	Top-5: Accuracy = 86.16%
"""

"""
мобильные телефоны
	офферов 607 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 46.46%
	Top-3: Accuracy = 92.26%
	Top-5: Accuracy = 93.08%
"""

"""
colbert-196-finish_offers_top_n_Apple_test_bs200.csv
мобильные телефоны
	офферов 3356 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 89.18%
	Top-3: Accuracy = 95.29%
	Top-5: Accuracy = 95.35%
"""

"""
colbert-196-finish_lr05_offers_top_n_Apple_test_bs200.csv
мобильные телефоны
	офферов 3356 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 86.41%
	Top-3: Accuracy = 97.79%
	Top-5: Accuracy = 98.36%
"""

"""
colbert-196-finish_amp=False_offers_top_n_Apple_test_bs200.csv
мобильные телефоны
	офферов 3356 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 89.18%
	Top-3: Accuracy = 95.29%
	Top-5: Accuracy = 95.35%
"""

"""
colbert-187-finish_accumsteps=10_offers_top_n_Apple_test.csv
мобильные телефоны
	офферов 3356 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 87.13%
	Top-3: Accuracy = 98.63%
	Top-5: Accuracy = 98.87%
"""

"""
colbert-18-finish_accumsteps=10_bsize_2100_offers_top_n_Apple_test.csv
мобильные телефоны
	офферов 3356 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 83.34%
	Top-3: Accuracy = 96.16%
	Top-5: Accuracy = 97.88%
"""

"""
2801_bsize210_offers_top_n_Apple_test.csv
мобильные телефоны
	офферов 3356 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 89.99%
	Top-3: Accuracy = 97.94%
	Top-5: Accuracy = 98.09%
"""

'\n2801_bsize210_offers_top_n_Apple_test.csv\nмобильные телефоны\n\tофферов 3356 шт поиск матча для каждого среди 698 шт моделей:\n\n\tTop-1: Accuracy = 89.99%\n\tTop-3: Accuracy = 97.94%\n\tTop-5: Accuracy = 98.09%\n'

In [10]:
def write_errors(df_offers, df, df_models, dst_fld, id_category):
    os.makedirs(dst_fld, exist_ok=True)
    for category in id_category.values():
        category_error = df[df.category_name == category]
        category_all = df_offers[df_offers.category_name == category]
        accuracies_top_n = calc_acc_for_category(category_error, df_models, category)
        with open(os.path.join(dst_fld, f"{category}.txt"), "a") as txt:
            txt.write(f'Всего офферов {len(category_all)} шт из них ошибок {len(category_error)} шт\nРаспределение ошибок:\n')
            txt.write(f'\nTop-1: Accuracy = {accuracies_top_n[0]:.2%}')
            txt.write(f'\nTop-2: Accuracy = {accuracies_top_n[1]:.2%}')
            txt.write(f'\nTop-3: Accuracy = {accuracies_top_n[2]:.2%}')
            txt.write(f'\nTop-4: Accuracy = {accuracies_top_n[3]:.2%}')
            txt.write(f'\nTop-5: Accuracy = {accuracies_top_n[4]:.2%}\n\n')
            txt.write("Истинная модель:\nПоле name оффера --> Найденная модель топ1")
            txt.write("\n____________________________________________________________\n")
            for index, row in category_error.iterrows():
                model_gt = list(df_models[df_models.model_id == row['model_id']]['full_name'])[0]
                model_pred = list(df_models[df_models.model_id == row['model_id_pred_1']]['full_name'])[0]
                txt.write(f"\n{model_gt}:\n(offer_id={row['offer_id']}): {row['name']} --> {model_pred}\n")                
def exclude_from_df(df, col):
    return df[df['model_id'] != df[col]].reset_index(drop=True)

clms = ['model_id_pred_1','model_id_pred_2','model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
df = df_offers.copy()
for i in range(len(clms)):
    df = exclude_from_df(df, clms[i])
    dst_fld = f"/home/sondors/Documents/price/ColBERT/EVAL/errors_ColBERT/exclude_top_{i+1}"
    write_errors(df_offers, df, df_models, dst_fld, id_category)

мобильные телефоны
	офферов 336 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 79.46%
	Top-5: Accuracy = 80.95%

мобильные телефоны
	офферов 85 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 18.82%
	Top-5: Accuracy = 24.71%

мобильные телефоны
	офферов 69 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 7.25%

мобильные телефоны
	офферов 67 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 4.48%

мобильные телефоны
	офферов 64 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 0.00%



In [5]:
df

,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,...,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_pred,similarity
0,Смартфон Apple iPhone 15 Pro 256 Гб Dual SIM т...,7883739,1876282775,True,мобильные телефоны,2801,135880.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Новый инновационный дизайн включает в себя зад...,Цифровая техника :: Смартфоны и мобильные теле...,...,7883741,26.91,7883733,26.88,7883736,25.14,7883640,23.86,"[7883737, 7883741, 7883733, 7883736, 7883640]","[29.88, 26.91, 26.88, 25.14, 23.86]"
1,"Смартфон Apple iPhone 13 A2631, 6.1"" 1170x2532...",7883324,1870923836,False,мобильные телефоны,2801,75500.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Артикул: 1104784 // Гарантия: 1 год | Прибытие...,Телефония :: Смартфоны,...,7883330,19.42,7883329,19.33,7883319,19.27,7883336,19.27,"[7883331, 7883330, 7883329, 7883319, 7883336]","[19.83, 19.42, 19.33, 19.27, 19.27]"
2,Смартфон Apple iPhone 15 Pro Max,7883786,1870801662,True,мобильные телефоны,2801,161788.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple MU6V3J/A,"Ноутбуки, планшеты, смартфоны :: Смартфоны",...,7883777,21.98,7883773,21.36,7883775,20.67,7883771,20.45,"[7883772, 7883777, 7883773, 7883775, 7883771]","[22.03, 21.98, 21.36, 20.67, 20.45]"
3,Смартфон Apple iPhone 14 256Gb Yellow (2 SIM)(...,7883442,1870405495,False,мобильные телефоны,2801,84140.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple iPhone 14 256Gb Yellow (2 SIM)(...,Портативная электроника :: Смартфоны,...,7883436,29.66,7883436,29.66,7883436,29.66,7883436,29.66,"[7883436, 7883436, 7883436, 7883436, 7883436]","[29.66, 29.66, 29.66, 29.66, 29.66]"
4,Смартфон Apple IPHONE 15 512GB GREEN Dual sim,7883658,1871034554,False,мобильные телефоны,2801,210990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",NaN,Смартфоны и гаджеты :: Смартфоны,...,7883652,24.89,7883651,23.95,7883654,23.61,7883655,23.03,"[7883653, 7883652, 7883651, 7883654, 7883655]","[30.02, 24.89, 23.95, 23.61, 23.03]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"Смартфон Apple iPhone 15 Pro Max A3106, 6.7"" 1...",7883787,1876370047,False,мобильные телефоны,2801,189700.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Артикул: 1267551 // Гарантия: 1 год | Прибытие...,Телефония :: Смартфоны,...,7883771,21.78,7883775,21.62,7883776,21.59,7883777,21.53,"[7883772, 7883771, 7883775, 7883776, 7883777]","[22.11, 21.78, 21.62, 21.59, 21.53]"
60,"Смартфон Apple iPhone 15 Pro Max A3106, 6.7"" 1...",7883786,1876370046,False,мобильные телефоны,2801,189700.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Артикул: 1267549 // Гарантия: 1 год | Прибытие...,Телефония :: Смартфоны,...,7883771,21.78,7883775,21.62,7883776,21.59,7883777,21.53,"[7883772, 7883771, 7883775, 7883776, 7883777]","[22.11, 21.78, 21.62, 21.59, 21.53]"
61,Смартфон Apple A3096 iPhone 15 Plus 128Gb черн...,7883671,1870284861,False,мобильные телефоны,2801,98100.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple A3096 iPhone 15 Plus 128Gb черн...,"Смартфоны, планшеты и гаджеты :: Смартфоны",...,7883661,26.00,7883474,20.14,7883482,18.16,7883482,18.16,"[7883666, 7883661, 7883474, 7883482, 7883482]","[28.48, 26.0, 20.14, 18.16, 18.16]"
62,Смартфон Apple iPhone 15 128Gb SIM + SIM Зеленый,7883625,1873611854,False,мобильные телефоны,2801,87990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","Аппарат не поддерживает eSIM, только 2 физичес...",Смартфоны :: Apple,...,7883658,24.66,7883629,22.88,7883642,22.78,7883191,22.30,"[7883630, 7883658, 7883629, 7883642, 7883191]","[31.45, 24.66, 22.88, 22.78, 22.3]"


In [6]:
offer1 = 'Смартфон Apple IPHONE 15 512GB GREEN Dual sim'
index = df.index[df['name'] == offer1][0]
dict(df.iloc[index])

{'name': 'Смартфон Apple IPHONE 15 512GB GREEN Dual sim',
 'model_id': 7883658,
 'offer_id': 1871034554,
 'modelized_by_barcode': False,
 'category_name': 'мобильные телефоны',
 'category_id': 2801,
 'price': 210990.0,
 'attrs': "[{'id': 3, 'type': 'e', 'value': 676.0}, {'id': 14528, 'type': 'b', 'value': 1.0}, {'id': 25534, 'type': 'e', 'value': 718.0}, {'id': 25586, 'type': 'e', 'value': 733.0}]",
 'description': nan,
 'external_category': 'Смартфоны и гаджеты :: Смартфоны',
 'external_brand': 'Apple',
 'external_type': 'Смартфон',
 'model_id_pred_1': 7883653,
 'similarity_1': 30.02,
 'model_id_pred_2': 7883652,
 'similarity_2': 24.89,
 'model_id_pred_3': 7883651,
 'similarity_3': 23.95,
 'model_id_pred_4': 7883654,
 'similarity_4': 23.61,
 'model_id_pred_5': 7883655,
 'similarity_5': 23.03,
 'model_id_pred': [7883653, 7883652, 7883651, 7883654, 7883655],
 'similarity': [30.02, 24.89, 23.95, 23.61, 23.03]}

In [7]:
# df.to_csv("/home/sondors/Documents/price/ColBERT_data/10_categories/740101/test/740101_model_id_not_in_top_5.csv", sep=';',index=False)